<a href="https://colab.research.google.com/github/its-rahul-cloud/MachineLearning/blob/main/Image_caption_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download shadabhussain/flickr8k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
100% 2.12G/2.13G [00:35<00:00, 60.6MB/s]
100% 2.13G/2.13G [00:35<00:00, 65.2MB/s]


In [ ]:
!unzip flickr8k.zip

#Imports

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from keras.layers import add
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import numpy as np
from PIL import Image




# Image Feature Extraction
**extract image feature by applying Resnet50 

**After executing the code, the output would be a 1x2048 vector for each image, containing the features from the images

Try can be made using another model then Resnet50 such as inseptionv3


In [4]:
%%time

#Get all filename of the image
folder='/content/flickr_data/Flickr_Data/Images/'
images=os.listdir(folder)

#Load the CNN Architecture with Imagenet and weight
image_model=ResNet50(weights='imagenet')
model_new=tf.keras.Model(image_model.input,image_model.layers[-2].output)

#Store image features in dictionary
img_features=dict()
for img in images:
  img1=image.load_img(folder+ img,target_size=(224,224))
  x = image.img_to_array(img1)
  x = np.expand_dims(x,axis=0)
  x = preprocess_input(x)
  fea_x= model_new.predict(x)
  fea_x1 = np.reshape(fea_x, fea_x.shape[1])
  img_features[img] = fea_x1






Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 32ms/step
CPU times: user 9min 46s, sys: 20 s, total: 10min 6s
Wall time: 11min 36s


In [5]:
from re import A
model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape= (224,224,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.Dense(2048,activation=tf.nn.softmax)
])


In [6]:
fn="/content/flickr_data/Flickr_Data/Flickr_TextData/Flickr8k.token.txt"
f=open(fn,'r')
capts=f.read()
#Group all caption by filename ,for references
captions=dict()
i=0

try:
  for line in capts.split("\n"):
    txt =line.split('\t')
    fn= txt[0].split('#')[0]
    if fn not in captions.keys():
        captions[fn]=[txt[1]]
    else :
      captions[fn].append(txt[1])
    i+=1
except:
  pass

In [7]:
def getCaptions(path):
    
    f = open(path, 'r')
    capts = f.read()
    desc = dict()

    try:
        for line in capts.split("\n"):
            image_id = line
            image_descs = captions[image_id]

            for des in image_descs:
                ws = des.split(" ")
                w = [word for word in ws if word.isalpha()]
                des = "startseq " + " ".join(w) + " endseq"
                if image_id not in desc:
                    desc[image_id] = list()
                desc[image_id].append(des)
    except:
        pass
    
    return desc

In [8]:
# Split captions
train_caps = getCaptions("/content/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt")
val_caps = getCaptions("/content/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.devImages.txt")

Some objects of train  and validation captions

In [9]:
print("train_caps type: ", type(train_caps))
print("First few (key, value) paris of train_caps:") 
for i, (k, v) in enumerate(train_caps.items()):
    print(k, v)
    if i>2:
        break
print("")
print("val_caps type: ", type(val_caps))
print("First few (key, value) pairs of val_caps:") 
for i, (k, v) in enumerate(val_caps.items()):
    print(k, v)
    if i>2:
        break

train_caps type:  <class 'dict'>
First few (key, value) paris of train_caps:
2513260012_03d33305cf.jpg ['startseq A black dog is running after a white dog in the snow endseq', 'startseq Black dog chasing brown dog through snow endseq', 'startseq Two dogs chase each other across the snowy ground endseq', 'startseq Two dogs play together in the snow endseq', 'startseq Two dogs running through a low lying body of water endseq']
2903617548_d3e38d7f88.jpg ['startseq A little baby plays croquet endseq', 'startseq A little girl plays croquet next to a truck endseq', 'startseq The child is playing croquette by the truck endseq', 'startseq The kid is in front of a car with a put and a ball endseq', 'startseq The little boy is playing with a croquet hammer and ball beside the car endseq']
3338291921_fe7ae0c8f8.jpg ['startseq A brown dog in the snow has something hot pink in its mouth endseq', 'startseq A brown dog in the snow holding a pink hat endseq', 'startseq A brown dog is holding a pink sh

#Tokenization

providing every word in train data some kind of token to recoginize it.





In [10]:
#Preparing to makeword-index and index-word
#(adding all training captions to list)
train_captions=[]
for key,desc_list in train_caps.items():
  for i in range(len(desc_list)):
    train_captions.append(desc_list[i])

#  Tokenize top 5000 words in Train captions
vocab_size=5000
tokenizer=Tokenizer( num_words=vocab_size,
                     oov_token="<unk>",
                     filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
word_index=tokenizer.word_index
index_word=tokenizer.index_word 

# Tokenizer index

In [11]:
vocab_size=len(tokenizer.word_index)+1
print(vocab_size)

7278


In [12]:
for i, (k, v) in enumerate(word_index.items()):
    print(k, v)
    if i>2:
        break
print("")
print("index_word type: ", type(index_word))
print("First few (key, value) pairs of index_word:") 
for i, (k, v) in enumerate(index_word.items()):
    print(k, v)
    if i>2:
        break

<unk> 1
a 2
startseq 3
endseq 4

index_word type:  <class 'dict'>
First few (key, value) pairs of index_word:
1 <unk>
2 a
3 startseq
4 endseq


Images and image features

In [19]:
%%time

train_fns=list(train_caps.keys())
train_set=dict((k,img_features[k]) for k in train_fns)
val_fns=list(val_caps.keys())
val_set=dict((k,img_features[k])for k in val_fns)
fn_test='/content/flickr_data/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'
f=open(fn_test,'r')
t=f.read()

test_fns= t.split("\n")
test_set=dict((k,img_features[k])for k in list(test_fns[:-1]))



CPU times: user 6.2 ms, sys: 0 ns, total: 6.2 ms
Wall time: 6.01 ms


In [20]:
#create sequences of images ,input sequences and output_words for an image
def create_sequences(tokenizer,max_length,desc_list ,photo,vocab_size):
  X1,X2,y=list(),list(),list()
  #walk through each discription for the image
  for desc in desc_list:
    #encode the sequence
    seq= tokenizer.texts_to_sequences([desc])[0]
    #split one sequence into mutiple  X,y pairs
    for i in range(1,len(seq )):
      #split into input and output pairs
      in_seq,out_seq=seq[:i],seq[i]
      #pad input sequence
      in_seq = pad_sequences([in_seq],maxlen=max_length)[0]

      #encode output sequence 
      out_seq=to_categorical([out_seq],num_classes=vocab_size)

      #store 
      X1.append(photo)
      X2.append(in_seq)
      y.append(out_seq)
    return np.array(X1) ,np.array(X2),np.array(y)
    




In [21]:
max_length=34


In [22]:
#data generator ,intended to be used in a call to model.fit_genrator
def data_generator(descriptions,photos,tokenizer,max_length,vocab_size):
  #loop for ever over the image
  while 1:
    for key, desc_list in descriptions.items():
      #retrive  the photo features
      photo =photos[key]
      in_img,in_seq,out_word=create_sequences(tokenizer,max_length,desc_list,photo,vocab_size)
      yield[in_img,in_seq],out_word


In [23]:
# test the data generator
generator = data_generator(train_caps, train_set, tokenizer, max_length, vocab_size)
inputs, outputs = next(generator)
print(inputs[0].shape)
print(inputs[1].shape)
print(outputs.shape)

(13, 2048)
(13, 34)
(13, 1, 7278)


# Word Embedding Matrix
we are going to use GLove Word Embedding for the captions.

In [24]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-10-19 10:43:53--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-10-19 10:43:53--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-10-19 10:43:54--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [25]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [30]:
%%time

#load Glove vectors
embeddings_index={}
f=open('/content/glove.6B.200d.txt',encoding='utf-8')
for line in f:
  values=line.split()
  word= values[0]
  coefs= np.asarray(values[1:],dtype='float32')
  embeddings_index[word]=coefs
f.close()


print('Found %s word vectors.'% len (embeddings_index))
# Get 200 -dim dense vector for each   of the 10000 words in out vocubulary
vocab_size=len(word_index)+1
embedding_dim= 200
embedding_matrix=np.zeros((vocab_size,embedding_dim))

for word ,i in word_index.items():

  embedding_vector=embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i]=embedding_vector


Found 400000 word vectors.
CPU times: user 12.9 s, sys: 821 ms, total: 13.7 s
Wall time: 13.5 s


# Architecture of model

*  we are going create two model 
     * image model (Resnet50)with some exam layers
     * language model (Embedding layers using glove 200d )
* In last concatination of both model
   * For the next-word prediction part, there are 2 layers of LSTM followed by a Dense and Softmax layer for classification 





In [56]:
from tensorflow.keras.layers import Dense,Embedding,LSTM,Concatenate
from keras.layers import Input, ConvLSTM2D, RepeatVector, Conv2D,TimeDistributed

In [74]:
image_model = tf.keras.models.Sequential()

image_model.add(Dense(embedding_dim, input_shape=(4096,), activation='relu'))
image_model.add(RepeatVector(max_length))

language_model = tf.keras.models.Sequential()

language_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_dim)))

conca = Concatenate()([image_model.output, language_model.output])
x = LSTM(128, return_sequences=True)(conca)
x = LSTM(512, return_sequences=False)(x)
x = Dense(vocab_size)(x)
out = Activation('softmax')(x)
model_1 = Model(inputs=[image_model.input, language_model.input], outputs = out)

model_1.layers[2].set_weights([embedding_matrix])
model_1.layers[2].trainable = False

model_1.compile(loss='categorical_crossentropy', optimizer = Adam(learning_rate = 0.0001), metrics=['accuracy'])

#Training

In [ ]:
%%time

# train the model, run epochs manually and save after each epoch
epochs = 10
steps = len(train_caps)
for i in range(epochs):
    # create the data generator
    generator = data_generator(train_caps, train_set, tokenizer, max_length, vocab_size)
    # fit for one epoch
    model_1.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)


Error becasue the colab is trying to change the model in graph model

# Inference


```
ALgo:=
For Each Image
KeepAddingWords = True
While KeepAddingWords:
yhat = model.predict([image_features,caption_sequence_so_far])
if yhat is Endseq:
KeepAddingWords = False
Else:
append yhat to caption_sequence_so_far

```



In [63]:
# map an integer to a word
def word_for_id(integer, tokenizer):
    return tokenizer.index_word.get(integer)

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        #print("sequence after tok: ", sequence)
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        if i==0:
            photo = np.expand_dims(photo, axis=0)
        #print("photo: ", photo)
        #print("sequence: ", sequence)
        yhat = model.predict([photo, sequence], verbose=0)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
    return in_text

In [66]:
def image_to_feat_vec(imagePath):
    img1 = image.load_img(imagePath, target_size=(224, 224))
    x = image.img_to_array(img1)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    fea_x = model_new.predict(x)
    fea_x1 = np.reshape(fea_x , fea_x.shape[1])
    return fea_x1


In [67]:
# test the predict function
imagePath = "/content/Flickr_Data/Flickr_Data/Images/1000268201_693b08cb0e.jpg"
photo = image_to_feat_vec(imagePath)
tmpimg1 = np.expand_dims(np.array(photo), axis=0)
print(tmpimg1.shape)
tmpcap1 = pad_sequences([[3]], maxlen=max_length)
print(tmpcap1.shape)
tmpout1 = model_1.predict([tmpimg1, tmpcap1], verbose=0)
print(tmpout1.shape)
print(tmpout1)

1/1 [==============================] - 0s 33ms/step
(1, 2048)
(1, 34)
(1, 7278)
[[0.00013795 0.00013762 0.0001464  ... 0.00014018 0.00014291 0.00014921]]
